<a href="https://colab.research.google.com/github/ibacaraujo/deeper/blob/master/deeper2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepER

### Install packages

In [4]:
%%shell
pip uninstall scikit-learn
pip install scikit-learn==0.21

Uninstalling scikit-learn-0.22.2.post1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scikit_learn-0.22.2.post1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 6.6MB 1.3MB/s 


In [1]:
%%shell
pip install py-entitymatching

In [2]:
%%shell

pip install fasttext

     |████████████████████████████████| 61kB 1.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2387325 sha256=f6194c942a137c343c30d63ecb78ecb9f9732ce244876cd0af532a62e0937509
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [3]:
%%shell

pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=c77956c46452f051765c5bfa33bfbe90bb810c03f175faad5907a1324f52600d
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


### Mount Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [24]:
%%shell
ls "/content/drive/My Drive"

'Colab Notebooks'   data-science-consultancy


### Configs

In [0]:
DATASET_ROOT = "/content/drive/My Drive/data-science-consultancy/BenchmarkDatasets/"
#Note: This contains also the label data!
CANDSET_IDS_FILE_NAME = "candset_ids_only.csv"

#TODO: Change me
CANDSET_FILE_NAME = "candset.pkl.compress"

#Size of distributed representations
#DR_DIMENSION = 300
FASTTEXT_MODEL_PATH = "/content/drive/My Drive/data-science-consultancy/fasttext/cc.pt.300.bin"

#Assumptions: partially for interoperability with Magellan/py_entitymatching
# 1. id column is always named as "id"
# 2. all relevant attributes have the same name. any columns without same names will be ignored

er_dataset_details = {
        "Abt_Buy" : {
            "dataset_folder_path" : DATASET_ROOT + "Abt_Buy/",
            "ltable_file_name" : "Abt.csv",
            "rtable_file_name" : "Buy.csv",
            "golden_label_file_name" : "abt_buy_perfectMapping.csv",
        },

        "Amazon_GoogleProducts" : {
            "dataset_folder_path" : DATASET_ROOT + "Amazon_GoogleProducts/",
            "ltable_file_name" : "Amazon.csv",
            "rtable_file_name" : "GoogleProducts.csv",
            "golden_label_file_name" : "Amzon_GoogleProducts_perfectMapping.csv",
        },

        "Walmart_Amazon" : {
            "dataset_folder_path" : DATASET_ROOT + "Walmart_Amazon/",
            "ltable_file_name" : "walmart.csv",
            "rtable_file_name" : "amazon.csv",
            "golden_label_file_name" : "matches_walmart_amazon.csv",
        },

        "DBLP_ACM" : {
            "dataset_folder_path" : DATASET_ROOT + "DBLP_ACM/",
            "ltable_file_name" : "DBLP2.csv",
            "rtable_file_name" : "ACM.csv",
            "golden_label_file_name" : "DBLP-ACM_perfectMapping.csv",
        },

        "DBLP_Scholar" : {
            "dataset_folder_path" : DATASET_ROOT + "DBLP_Scholar/",
            "ltable_file_name" : "DBLP1.csv",
            "rtable_file_name" : "Scholar.csv",
            "golden_label_file_name" : "DBLP-Scholar_perfectMapping.csv",
        },

        "Cora" : {
            "dataset_folder_path" : DATASET_ROOT + "Cora/",
            "ltable_file_name" : "cora.csv",
            "rtable_file_name" : "cora.csv",
            "golden_label_file_name" : "matches_cora.csv",
        },

        "Fodors_Zagat" : {
            "dataset_folder_path" : DATASET_ROOT + "Fodors_Zagat/",
            "ltable_file_name" : "fodors.csv",
            "rtable_file_name" : "zagats.csv",
            "golden_label_file_name" : "matches_fodors_zagats.csv",
        },

}

### Blocking utils

In [0]:
import py_entitymatching as em
import csv
import pandas as pd
import numpy as np

def load_known_dataset(dataset_name):
    dataset_dtls = er_dataset_details[dataset_name]
    l_file_name = dataset_dtls["dataset_folder_path"] + dataset_dtls["ltable_file_name"]
    r_file_name = dataset_dtls["dataset_folder_path"] + dataset_dtls["rtable_file_name"]

    #Assumption: key name is always "id"
    A = em.read_csv_metadata(l_file_name , key="id", encoding='utf-8')
    B = em.read_csv_metadata(r_file_name , key="id", encoding='utf-8')

    return A, B

#Given a dataset name, it returns all duplicates as a Pandas DF
#Assumes the first row is a header
def get_all_duplicates_as_df(dataset_name):
    dataset_dtls = er_dataset_details[dataset_name]
    duplicates_file_name = dataset_dtls["dataset_folder_path"] + dataset_dtls["golden_label_file_name"]
    duplicates_df = pd.read_csv(duplicates_file_name)
    return duplicates_df

#Given a dataset name and candidate set df, this adds a column called gold.
# it is set to  1 for all duplicates and 0 for others
def add_labels_to_candset(dataset_name, candset_df, objectify=False):
    duplicates_df = get_all_duplicates_as_df(dataset_name)
    #rename columns to make merging easier
    duplicates_df.columns = ["ltable_id", "rtable_id"]

    #Sometimes pandas / Magellan puts some columns as objects instead of numeric/string. In this case, we will force this to join appropriately
    if objectify:
        duplicates_df = duplicates_df.astype(object)

    #We merged two DF based on the common attributes. The indicator 'gold' takes three values both, left_only, right_only
    df_with_gold = pd.merge(candset_df, duplicates_df, on=['ltable_id', 'rtable_id'], how='left', indicator='gold')
    #If it is present in both, then it is a duplicate and we set it to 1 and 0 otherwise
    df_with_gold['gold'] = np.where(df_with_gold.gold == 'both', 1, 0)

    #print df_with_gold.loc[df_with_gold['gold']==1,['ltable_id', 'rtable_id']]
    return df_with_gold

def save_candset_ids_only(dataset_name, candset_df):
    dataset_dtls = er_dataset_details[dataset_name]
    output_file_name = dataset_dtls["dataset_folder_path"] + CANDSET_IDS_FILE_NAME
    candset_df[ ['ltable_id', 'rtable_id', 'gold'] ].to_csv(output_file_name, index=False)

def save_candset_compressed(dataset_name, candset_df, file_name):
    dataset_dtls = er_dataset_details[dataset_name]
    output_file_name = dataset_dtls["dataset_folder_path"] + file_name
    candset_df.to_pickle(output_file_name, compression="gzip")

#This function takes the dataset name and block_fn is a function pointer for each dataset used for its blocking
# This stores both the candset in a compressed file and also the ids of candset
def save_candset_wrapper(dataset_name, block_fn, objectify=False):
    A, B = load_known_dataset(dataset_name)
    candset_df = block_fn(A, B)
    candset_with_labels_df = add_labels_to_candset(dataset_name, candset_df, objectify)
    save_candset_compressed(dataset_name, candset_with_labels_df, CANDSET_FILE_NAME)
    save_candset_ids_only(dataset_name, candset_with_labels_df)

def load_candset_compressed(dataset_name, file_name):
    dataset_dtls = er_dataset_details[dataset_name]
    input_file_name = dataset_dtls["dataset_folder_path"] + file_name

    #Magellan complains of missing keys if this is not done!
    ltable, rtable = load_known_dataset(dataset_name)

    #Load compressed file and set Magellan's properties
    candset_df = pd.read_pickle(input_file_name, compression="gzip")

    em.set_key(candset_df,'_id')
    em.set_property(candset_df,'fk_ltable',"ltable_id")
    em.set_property(candset_df,'fk_rtable',"rtable_id")
    em.set_property(candset_df,'ltable',ltable)
    em.set_property(candset_df,'rtable',rtable)

    return candset_df

#assumes the names of foreign keys as ltable_id and rtable_id
def verify_blocking_ground_truth(dataset_name, block_df, objectify=False):
    dataset_dtls = er_dataset_details[dataset_name]
    all_matches_csv_file_name = dataset_dtls["dataset_folder_path"] + dataset_dtls["golden_label_file_name"]
    duplicates_df = pd.read_csv(all_matches_csv_file_name)

    num_duplicates_missed = 0
    duplicates_df.columns = ["ltable_id", "rtable_id"]
    #Sometimes pandas / Magellan puts some columns as objects instead of numeric/string. In this case, we will force this to join appropriately
    if objectify:
        duplicates_df = duplicates_df.astype(object)

    #Intuition: merge function joints two data frames. The outer option creates a number of NaN rows when
    # some duplicates are missing in the blocked_df
    # we leverage the fact that len gives all rows while count gives non-NaN to compute the missing options
    merged_df = block_df.merge(duplicates_df, left_on=["ltable_id", "rtable_id"], right_on=["ltable_id", "rtable_id"], how='outer')
    num_duplicates_missed = len(merged_df) - merged_df["_id"].count()
    total_duplicates = len(duplicates_df)

    print("Size of candset =", len(block_df))
    print("Totally missed:", num_duplicates_missed, " out of ", total_duplicates)


def block_fodors_zagats(A, B):
    ob = em.OverlapBlocker()
    #No misses
    C = ob.block_tables(A, B, 'name', 'name', l_output_attrs=['name', 'addr', 'city', 'phone'],  r_output_attrs=['name', 'addr', 'city', 'phone'],
            overlap_size=1, show_progress=False)
    return C


def block_dblp_acm(A, B):
    ab = em.AttrEquivalenceBlocker()
    C = ab.block_tables(A, B, l_block_attr='year', r_block_attr='year', l_output_attrs=["title","authors","venue","year"],
		r_output_attrs=["title","authors","venue","year"], allow_missing=True)

    ob = em.OverlapBlocker()

    #=================>results in a candidate set of size 200K with no missing duplicates out of 2224
    #C1 = ob.block_candset(C, 'title', 'title', word_level=True, overlap_size=1, show_progress=True)

    #=================>results in a candidate set of size 46K with 5 missing duplicates out of 2224
    C2 = ob.block_candset(C, 'title', 'title', word_level=True, overlap_size=2, show_progress=True)
    return C2


def block_amazon_googleproducts(A, B):
    ob = em.OverlapBlocker()
    #=================>results in a candidate set of size 400K with 6 missing duplicates out of 1300
    C = ob.block_tables(A, B, "title", "title", word_level=True, overlap_size=1, l_output_attrs=["title","description","manufacturer","price"], r_output_attrs=["title","description","manufacturer","price"], show_progress=True, allow_missing=True)
    return C


def block_abt_buy(A, B):
    B["description"] = B["description"] + " " + B["manufacturer"]
    ob = em.OverlapBlocker()
    #=================>results in a candidate set of size 164K with 6 missing duplicates out of 1097
    C = ob.block_tables(A, B, "name", "name", word_level=True, overlap_size=1,
	l_output_attrs=["name","description","price"], r_output_attrs=["name","description","price"], show_progress=True, allow_missing=True)
    return C

def block_walmart_amazon(A, B):
    #assumes some preprocessing is done:
    #Specifically in amazon.csv : a.	pcategory2  => groupname , b.	{ proddescrshort,proddescrlong } => shortdescr,longdescr

    ob = em.OverlapBlocker()

    #C1 = ob.block_tables(ltable, rtable, 'title', 'title', word_level=True, overlap_size=2)
    #=================>results in a candidate set of size 1.1M with 20 missing duplicates out of 1154
    #blocking_utils.verify_blocking_ground_truth(dataset_name, C1)

    attributes = ['brand', 'groupname', 'title', 'price', 'shortdescr', 'longdescr', 'imageurl', 'modelno', 'shipweight', 'dimensions']
    C2 = ob.block_tables(A, B, 'title', 'title', word_level=True, overlap_size=3, l_output_attrs=attributes, r_output_attrs=attributes)
    #=================>results in a candidate set of size 278K with 84 missing duplicates out of 1154
    #blocking_utils.verify_blocking_ground_truth(dataset_name, C2)

    return C2

def block_dblp_scholar(A, B):
    ob = em.OverlapBlocker()
    attributes = ["id","title","authors","venue","year"]
    #C1 = ob.block_tables(A, B, 'title', 'title', word_level=True, overlap_size=3, show_progress=True, l_output_attrs=attributes, r_output_attrs=attributes)
    #=================>results in a candidate set of size 1.2M with 178 missing duplicates out of 5347
    C2 = ob.block_tables(A, B, 'title', 'title', word_level=True, overlap_size=4, show_progress=True, l_output_attrs=attributes, r_output_attrs=attributes)
    #=================>results in a candidate set of size 135K with 467 missing duplicates out of 5347
    return C2

### LSH

In [4]:
'''
Copyright (c) 2011, Yahoo! Inc.
All rights reserved.

Redistribution and use of this software in source and binary forms,
with or without modification, are permitted provided that the following
conditions are met:

* Redistributions of source code must retain the above
  copyright notice, this list of conditions and the
  following disclaimer.

* Redistributions in binary form must reproduce the above
  copyright notice, this list of conditions and the
  following disclaimer in the documentation and/or other
  materials provided with the distribution.

* Neither the name of Yahoo! Inc. nor the names of its
  contributors may be used to endorse or promote products
  derived from this software without specific prior
  written permission of Yahoo! Inc.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS
IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED
TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
'''

# Python implementation of Andoni's e2LSH.  This version is fast because it
# uses Python hashes to implement the buckets.  The numerics are handled
# by the numpy routine so this should be close to optimal in speed (although
# there is no control of the hash tables layout in memory.)

# This file implements the following classes
#	lsh - the basic projection algorithm (on k-dimensional hash)
#	index - a group of L lsh hashes
#	TestDataClass - a generic class for handling the raw data

# To use
#	Call this routine with the -histogram flag to create some random
#		test data and to compute the nearest-neighbor distances
#	Load the .distance file that is produced into Matlab and compute
#		the d_nn and d_any histograms from the first and second columns
#		of the .distance data.
#	Use these histograms (and their bin positions) as input to the
#		Matlab ComputeMPLSHParameters() routine.
#	This gives you the optimum LSH parameters.  You can use these
#		values directly as parameters to this code.
#	You can use the -ktest, -ltest and -wtest flags to test the
#		parameters.

# Prerequisites: Python version 2.6 (2.5 might work) and NumPy

# By Malcolm Slaney, Yahoo! Research

import random, numpy, pickle, os, operator, traceback, sys, math, time
import itertools					# For Multiprobe

#######################################################################
# Note, the data is always a numpy array of size Dx1.
#######################################################################

# This class just implements k-projections into k integers
# (after quantization) and then reducing that integer vector
# into a T1 and T2 hash.  Data can either be entered into a
# table, or retrieved.

class lsh:
	'''This class implements one k-dimensional projection, the T1/T2 hashing
	and stores the results in a table for later retrieval.  Input parameters
	are the bin width (w, floating point, or float('inf') to get binary LSH),
	and the number of projections to compute for one table entry (k, an integer).'''
	def __init__(self, w, k):
		self.k = k	# Number of projections
		self.w = w	# Bin width
		self.projections = None
		self.buckets = {}

	# This only works for Python >= 2.6
	def sizeof(self):
		'''Return how much storage is needed for this object. In bytes
		'''
		return sys.getsizeof(self.buckets) + \
			sys.getsizeof(self.projections) + \
			sys.getsizeof(self)

	# Create the random constants needed for the projections.
	# Can't do this until we see some data, so we know the
	# diementionality.
	def CreateProjections(self, dim):
		self.dim = dim
		# print "CreateProjections: Creating projection matrix for %dx%d data." % (self.k, self.dim)
		self.projections = numpy.random.randn(self.k, self.dim)
		self.bias = numpy.random.rand(self.k, 1)
		if 0:
			print("Dim is", self.dim)
			print('Projections:\n', self.projections)
			# print 'T1 hash:\n', self.t1hash
			# print 'T2 hash:\n', self.t2hash
		if 0:
			# Write out the project data so we can check it's properties.
			# Should be Gaussian with mean of zero and variance of 1.
			fp = open('Projections.data', 'w')
			for i in xrange(0,self.projections.shape[0]):
				for j in xrange(0,self.projections.shape[1]):
					fp.write('%g ' % self.projections[i,j])
				fp.write('\n')

	# Compute the t1 and t2 hashes for some data.  Doing it this way
	# instead of in a loop, as before, is 10x faster.  Thanks to Anirban
	# for pointing out the flaw.  Not sure if the T2 hash is needed since
	# our T1 hash is so strong.
	debugFP = None
	firstTimeCalculateHashes = False		# Change to false to turn this off
	infinity = float('inf')					# Easy way to access this flag
	def CalculateHashes(self, data):
		'''Multiply the projection data (KxD) by some data (Dx1),
		and quantize'''
                #Sara: changed self.projections == None to self.projections is None
		if self.projections is None:
			self.CreateProjections(len(data))
		bins = numpy.zeros((self.k,1), 'int')
		if lsh.firstTimeCalculateHashes:
			print('data = ', numpy.transpose(data))
			print('bias = ', numpy.transpose(self.bias))
			print('projections = ',)
			for i in range(0, self.projections.shape[0]):
				for j in range(0, self.projections.shape[1]):
					print(self.projections[i][j]),
				print()
			# print 't1Hash = ', self.t1hash
			# print 't2Hash = ', self.t2hash
			firstTimeCalculateHashes = False
			print("Bin values:", self.bias + \
							numpy.dot(self.projections, data)/self.w)
			print("Type of bins:", type(self.bias + \
							numpy.dot(self.projections, data)/self.w))
		if 0:
			if lsh.debugFP == None:
				print("Opening Projections file")
				lsh.debugFP = open('Projections.data', 'w')
			d = self.bias + numpy.dot(self.projections, data)/self.w
			for i in xrange(0, len(d)):
				lsh.debugFP.write('%g\n' % d[i])
			lsh.debugFP.write('\n')
			lsh.debugFP.flush()
		if self.w == lsh.infinity:
			# Binary LSH
			bins[:] = (numpy.sign(numpy.dot(self.projections, data))+1)/2.0
		else:
			bins[:] = numpy.floor(self.bias + numpy.dot(self.projections, data)/self.w)
		t1 = self.ListHash(bins)
		t2 = self.ListHash(bins[::-1])		# Reverse data for second hash
		return t1, t2

	# Input: A Nx1 array (of integers)
	# Output: A 28 bit hash value.
	# From: http://stackoverflow.com/questions/2909106/
	#	python-whats-a-correct-and-good-way-to-implement-hash/2909572#2909572
	def ListHash(self, d):
		# return str(d).__hash__()		# Good for testing, but not efficient
                #Sara: changed d == None to d is None
		if d is None or len(d) == 0:
			return 0
		# d = d.reshape((d.shape[0]*d.shape[1]))
		value = d[0, 0] << 7
		for i in d[:,0]:
			value = (101*value + i)&0xfffffff
		return value

	# Just a debug version that returns the bins too.
	def CalculateHashes2(self, data):
		if self.projections == None:
			print("CalculateHashes2: data.shape=%s, len(data)=%d" % (str(data.shape), len(data)))
			self.CreateProjections(len(data))
		bins = numpy.zeros((self.k,1), 'int')
		parray = numpy.dot(self.projections, data)
		bins[:] = numpy.floor(parray/self.w  + self.bias)
		t1 = self.ListHash(bins)
		t2 = self.ListHash(bins[::-1])		# Reverse data for second hash
		# print self.projections, data, parray, bins
		# sys.exit(1)
		return t1, t2, bins, parray

	# Return a bunch of hashes, depending on the level of multiprobe
	# asked for.  Each list entry contains T1, T2. This is a Python
	# iterator... so call it in a for loop.  Each iteration returns
	# a bin ID (t1,t2)
	# [Need to store bins in integer array so we don't convert to
	# longs prematurely and get the wrong hash!]
	def CalculateHashIterator(self, data, multiprobeRadius=0):
                #Sara: changed self.projections == None to self.projections is None
		if self.projections is None:
			self.CreateProjections(len(data))
		bins = numpy.zeros((self.k,1), 'int')
		directVector = numpy.zeros((self.k,1), 'int')
		newProbe = numpy.zeros((self.k,1), 'int')
		if self.w == lsh.infinity:
			points = numpy.dot(self.projections, data)
			bins[:] = (numpy.sign(points)+1)/2.0
			directVector[:] = -numpy.sign(bins-0.5)
		else:
			points = numpy.dot(self.projections, data)/self.w + self.bias
			bins[:] = numpy.floor(points)
			directVector[:] = numpy.sign(points-numpy.floor(points)-0.5)
		t1 = self.ListHash(bins)
		t2 = self.ListHash(bins[::-1])
		yield (t1,t2)
		if multiprobeRadius > 0:
			# print "Multiprobe points:", points
			# print "Multiprobe bin:", bins
			# print "Multiprobe direct:", directVector
			dimensions = range(self.k)
			deltaVector = numpy.zeros((self.k, 1), 'int')	# Preallocate
			for r in range(1, multiprobeRadius+1):
				# http://docs.python.org/library/itertools.html
				for candidates in itertools.combinations(dimensions, r):
					deltaVector *= 0						# Start Empty
					deltaVector[list(candidates), 0] = 1	# Set some bits
					newProbe[:] = bins + deltaVector*directVector	# New probe
					t1 = self.ListHash(newProbe)
					t2 = self.ListHash(newProbe[::-1])		# Reverse data for second hash
					# print "Multiprobe probe:",newProbe, t1, t2
					yield (t1,t2)

	# Put some data into the hash bucket for this LSH projection
	def InsertIntoTable(self, id, data):
		(t1, t2) = self.CalculateHashes(data)
		if t1 not in self.buckets:
			self.buckets[t1] = {t2: [id]}
		else:
			if t2 not in self.buckets[t1]:
				self.buckets[t1][t2] = [id]
			else:
				self.buckets[t1][t2].append(id)

	# Find some data in the hash bucket.  Return all the ids
	# that we find for this T1-T2 pair.
	def FindXXObsolete(self, data):
		(t1, t2) = self.CalculateHashes(data)
		if t1 not in self.buckets:
			return []
		row = self.buckets[t1]
		if t2 not in row:
			return []
		return row[t2]

	#
	def Find(self, data, multiprobeRadius=0):
		'''Find the points that are close to the query data.  Use multiprobe
		to also look in nearby buckets.'''
		res = []
		for (t1,t2) in self.CalculateHashIterator(data, multiprobeRadius):
			# print "Find t1:", t1
			if t1 not in self.buckets:
				continue
			row = self.buckets[t1]
			if t2 not in row:
				continue
			res += row[t2]
		return res

	# Create a dictionary showing all the buckets an ID appears in
	def CreateDictionary(self, theDictionary, prefix):
		for b in self.buckets:		# Over all buckets
			w = prefix + str(b)
			for c in self.buckets[b]:# Over all T2 hashes
				for i in self.buckets[b][c]:#Over ids
					if not i in theDictionary:
						theDictionary[i] = [w]
					else:
						theDictionary[i] += w
		return theDictionary


	# Print some stats for these lsh buckets
	def StatsXXX(self):
		maxCount = 0; sumCount = 0;
		numCount = 0; bucketLens = [];
		for b in self.buckets:
			for c in self.buckets[b]:
				l = len(self.buckets[b][c])
				if l > maxCount:
					maxCount = l
					maxLoc = (b,c)
					# print b,c,self.buckets[b][c]
				sumCount += l
				numCount += 1
				bucketLens.append(l)
		theValues = sorted(bucketLens)
		med = theValues[(len(theValues)+1)/2-1]
		print("Bucket Counts:")
		print("\tTotal indexed points:", sumCount)
		print("\tT1 Buckets filled: %d/%d" % (len(self.buckets), 0))
		print("\tT2 Buckets used: %d/%d" % (numCount, 0))
		print("\tMaximum T2 chain length:", maxCount, "at", maxLoc)
		print("\tAverage T2 chain length:", float(sumCount)/numCount)
		print("\tMedian T2 chain length:", med)

	def HealthStats(self):
		'''Count the number of points in each bucket (which is currently
		a function of both T1 and T2)'''
		maxCount = 0; numCount = 0; totalIndexPoints = 0;
		for b in self.buckets:
			for c in self.buckets[b]:
				l = len(self.buckets[b][c])
				if l > maxCount:
					maxCount = l
					maxLoc = (b,c)
					# print b,c,self.buckets[b][c]
				totalIndexPoints += l
				numCount += 1
		T1Buckets = len(self.buckets)
		T2Buckets = numCount
		T1T2BucketAverage = totalIndexPoints/float(numCount)
		T1T2BucketMax = maxCount
		return (T1Buckets, T2Buckets, T1T2BucketAverage, T1T2BucketMax)

	# Get a list of all IDs that are contained in these hash buckets
	def GetAllIndices(self):
		theList = []
		for b in self.buckets:
			for c in self.buckets[b]:
				theList += self.buckets[b][c]
		return theList

	# Put some data into the hash table, see how many collisions we get.
	def Test(self, n):
		self.buckets = {}
		self.projections = None
		d = numpy.array([.2,.3])
		for i in range(0,n):
			self.InsertIntoTable(i, d+i)
		for i in range(0,n):
			r = self.Find(d+i)
			matches = sum(map(lambda x: x==i, r))
			if matches == 0:
				print("Couldn't find item", i)
			elif matches == 1:
				pass
			if len(r) > 1:
				print("Found big bin for", i,":", r)


# Put together several LSH projections to form an index.  The only
# new parameter is the number of groups of projections (one LSH class
# object per group.)
class index:
	def __init__(self, w, k, l):
		self.k = k;
		self.l = l
		self.w = w
		self.projections = []
		self.myIDs = []
		for i in range(0,l):	# Create all LSH buckets
			self.projections.append(lsh(w, k))

	# Only works for Python > 2.6
	def sizeof(self):
		'''Return the sizeof this index in bytes.
		'''
		return sum(p.sizeof() for p in self.projections) + \
			sys.getsizeof(self)

	# Replace id we are given with a numerical id.  Since we are going
	# to use the ID in L tables, it is better to replace it here with
	# an integer.  We store the original ID in an array, and return it
	# to the user when we do a find().
	def AddIDToIndex(self, id):
		if type(id) == int:
			return id				# Don't bother if already an int
		self.myIDs.append(id)
		return len(self.myIDs)-1

	def FindID(self, id):
		if type(id) != int or id < 0 or id >= len(self.myIDs):
			return id
		return self.myIDs[id]

	# Insert some data into all LSH buckets
	def InsertIntoTable(self, id, data):
		intID = self.AddIDToIndex(id)
		for p in self.projections:
			p.InsertIntoTable(intID, data)

	def FindXXObsolete(self, data):
		'''Find some data in all the LSH buckets. Return a list of
		data's id and bucket counts'''
		items = [p.Find(data) for p in self.projections]
		results = {}
		for itemList in items:
			for item in itemList:
				if item in results:			# Much faster without setdefault
					results[item] += 1
				else:
					results[item] = 1
		s = sorted(results.items(), key=operator.itemgetter(1), \
			reverse=True)
		return [(self.FindID(i),c) for (i,c) in s]

	def Find(self, queryData, multiprobeR=0):
		'''Find some data in all the LSH tables.  Use Multiprobe, with
		the given radius, to search neighboring buckets.  Return a list of
		results.  Each result is a tuple consisting of the candidate ID
		and the number of times it was found in the index.'''
		results = {}
		for p in self.projections:
			ids = p.Find(queryData, multiprobeR)
			# print "Got back these IDs from p.Find:", ids
			for id in ids:
				if id in results:
					results[id] += 1
				else:
					results[id] = 1
		s = sorted(results.items(), key=operator.itemgetter(1), \
			reverse=True)
		return [(self.FindID(i),c) for (i,c) in s]

	def FindExact(self, queryData, GetData, multiprobeR=0):
		'''Return a list of results sorted by their exact
		distance from the query.  GetData is a function that
		returns the original data given its key.  This function returns
		a list of results, each result has the candidate ID and distance.'''
		s = self.Find(queryData, multiprobeR)
		# print "Intermediate results are:", s
		d = map(lambda id,count: (id,((GetData(id)-queryData)**2).sum(), \
				count), s)
		s = sorted(d, key=operator.itemgetter(1))
		return [(self.FindID(i),d) for (i,d,c) in s]

	# Put some data into the hash tables.
	def Test(self, n):
		d = numpy.array([.2,.3])
		for i in range(0,n):
			self.InsertIntoTable(i, d+i)
		for i in range(0,n):
			r = self.Find(d+i)
			print(r)

	# Print the statistics of each hash table.
	def Stats(self):
		for i in range(0, len(self.projections)):
			p = self.projections[i]
			print("Buckets", i),
			p.Stats()

	# Get al the IDs that are part of this index.  Just check one hash
	def GetAllIndices(self):
		if self.projections and len(self.projections) > 0:
			p = self.projections[0]
			return p.GetAllIndices()
		return None

	# Return the buckets (t1 and t2 hashes) associated with a data point
	def GetBuckets(self, data):
		b = []
		for p in self.projections:
			( t1, t2, bins, parray) = p.CalculateHashes2(data)
			print("Bucket:", t1, t2, bins, parray)
			b += (t1, t2)
		return b

	#
	def DictionaryPrefix(self, pc):
		prefix = 'W'
		prefixes = 'abcdefghijklmnopqrstuvwxyz'
		while pc > 0:	# Create unique ID for theis bucket
			prefix += prefixes[pc%len(prefixes)]
			pc /= len(prefixes)
		return prefix

	# Create a list ordered by ID listing which buckets are used for each ID
	def CreateDictionary(self):
		theDictionary = {}
		pi = 0
		for p in self.projections:
			prefix = self.DictionaryPrefix(pi)
			theDictionary = p.CreateDictionary(theDictionary,\
				prefix)
			pi += 1
		return theDictionary

	# Find the bucket ids that best correspond to this piece of data.
	def FindBuckets(self, data):
		theWords = []
		pi = 0
		for p in self.projections:
			prefix = self.DictionaryPrefix(pi)
			( t1, t2, bins, parray) = p.CalculateHashes2(data)
			word = prefix + str(t1)
			theWords += [word]
			pi += 1
		return theWords

# Save an LSH index to a pickle file.
def SaveIndex(filename, ind):
	try:
		fp = open(filename, 'w')
		pickle.dump(ind, fp)
		fp.close()
		statinfo = os.stat(filename,)
		if statinfo:
			print("Wrote out", statinfo.st_size, "bytes to", \
				filename)
	except:
		print("Couldn't pickle index to file", filename)
		traceback.print_exc(file=sys.stderr)

# Read an LSH index from a pickle file.
def LoadIndex(filename):
	if type(filename) == str:
		try:
			fp = open(filename, 'r')
		except:
			print("Couldn't open %s to read LSH Index" % (filename))
			return None
	else:
		fp = filename
	try:
		ind = pickle.load(fp)
		fp.close()
		return ind
	except:
		print("Couldn't read pickle file", filename)
		traceback.print_exc(file=sys.stderr)




class TestDataClass:
	'''A bunch of routines used to generate data we can use to test
	this LSH implementation.'''
	def __init__(self):
		self.myData = None
		self.myIndex = None
		self.nearestNeighbors = {}		# A dictionary pointing to IDs

	def LoadData(self, filename):
		'''Load data from a flat file, one line per data point.'''
		lineCount = 0
		try:
			fp = open(filename)
			if fp:
				for theLine in fp:			# Count lines in file
					if theLine == '':
						break
					lineCount += 1
				dim = len(theLine.split())	# Allocate the storage array
				self.myData = numpy.zeros((dim, lineCount))
				fp.seek(0,0)				# Go back to beginning of file
				lineCount = 0
				for theLine in fp:			# Now load the data
					data = [float(i) for i in theLine.split()]
					self.myData[:,lineCount] = data
					lineCount += 1
				fp.close()
			else:
				print("Can't open %s to LoadData()" % filename)
		except:
			print("Error loading data from %s in TestDataClass.LoadData()" \
				% filename)
			traceback.print_exc(file=sys.stderr)
		print("self.myData has %d lines and is:" % lineCount, self.myData)

	def SaveData(self, filename):
		'''Save this data in a flat file.  One line per data point.'''
		numDims = self.NumDimensions()
		try:
			fp = open(filename, 'w')
			if fp:
				for i in xrange(0, self.NumPoints()):
					data = self.RetrieveData(i).reshape(numDims)
					fp.write(' '.join([str(d) for d in data]) + '\n')
				fp.close()
				return
		except:
			pass
		sys.stderr.write("Can't write test data to %s\n" % filename)

	def CreateIndex(self, w, k, l):
		'''Create an index for the data we have in our database.  Inputs are
		the LSH parameters: w, k and l.'''
		self.myIndex = index(w, k, l)
		itemCount = 0
		tic = time.clock()
		for itemID in self.IterateKeys():
			features = self.RetrieveData(itemID)
			if features != None:
				self.myIndex.InsertIntoTable(itemID, features)
				itemCount += 1
		print("Finished indexing %d items in %g seconds." % \
			(itemCount, time.clock()-tic))
		sys.stdout.flush()

	def RetrieveData(self, id):
		'''Find a point in the array of data.'''
		id = int(id)						# Key in this base class is an int!
		if id < self.myData.shape[1]:
			return self.myData[:,id:id+1]
		return None

	def NumPoints(self):
		'''How many data point are in this database?'''
		return self.myData.shape[1]

	def NumDimensions(self):
		'''What is the dimensionality of the data?'''
		return self.myData.shape[0]

	def GetRandomQuery(self):
		'''Pick a random query from the dataset.  Return a key.'''
		return random.randrange(0,self.NumPoints())	# Pick random query

	def FindNearestNeighbors(self, count):
		'''Exhaustive search for count nearest-neighbor results.
		Save the results in a dictionary.'''
		numPoints = self.NumPoints()
		self.nearestNeighbors = {}
		for i in xrange(0,count):
			qid = self.GetRandomQuery()				# Pick random query
			qData = self.RetrieveData(qid)				# Find it's data
			nearestDistance2 = None
			nearestIndex = None
			for id2 in self.IterateKeys():
				if qid != id2:
					d2 = ((self.RetrieveData(id2)-qData)**2).sum()
					if id == -1:					# Debugging
						print(qid, id2, qData, self.RetrieveData(id2), d2)
					if nearestDistance2 == None or d2 < nearestDistance2:
						nearestDistance2 = d2
						nearestIndex = id2
			self.nearestNeighbors[qid] = \
				(nearestIndex, math.sqrt(nearestDistance2))
			if qid == -1:
				print(qid, nearestIndex, math.sqrt(nearestDistance2))
				sys.stdout.flush()

	def SaveNearestNeighbors(self, filename):
		'''Save the nearest neighbor dictionary in a file.  Each line
		of the file contains the query key, the distance to the nearest
		neighbor, and the NN key.'''
		if filename.endswith('.gz'):
			import gzip
			fp = gzip.open(filename, 'w')
		else:
			fp = open(filename, 'w')
		if fp:
			for (query,(nn,dist)) in self.nearestNeighbors.items():
				fp.write('%s %g %s\n' % (str(query), dist, str(nn)))
			fp.close()
		else:
			print("Can't open %s to write nearest-neighbor data" % filename)

	def LoadNearestNeighbors(self, filename):
		'''Load a file full of nearest neighbor data.'''
		self.nearestNeighbors = {}
		if filename.endswith('.gz'):
			import gzip
			fp = gzip.open(filename, 'r')
		else:
			fp = open(filename, 'r')
		if fp:
			print("Loading nearest-neighbor data from:", filename)
			for theLine in fp:
				(k,d,nn) = theLine.split()
				if type(self.myData) == numpy.ndarray: # Check for array indices
					k = int(k)
					nn = int(nn)
					if k < self.NumPoints() and nn < self.NumPoints():
						self.nearestNeighbors[k] = (nn,float(d))
				elif k in self.myData and nn in self.myData:	# dictionary index
					self.nearestNeighbors[k] = (nn,float(d))
			fp.close()
			print(" Loaded %d items into the nearest-neighbor dictionary." % len(self.nearestNeighbors))
		else:
			print("Can't open %s to read nearest neighbor data." % filename)

	def IterateKeys(self):
		'''Iterate through all possible keys in the dataset.'''
		for i in range(self.NumPoints()):
			yield i

	def FindMedian(self):
		numDim = self.NumDimensions()
		numPoints = self.NumPoints()
		oneColumn = numpy.zeros((numPoints))
		medians = numpy.zeros((numDim))
		for d in xrange(numDim):
			rowNumber = 0
			for k in self.IterateKeys():
				oneData = self.RetrieveData(k)
				oneColumn[rowNumber] = oneData[d]
				rowNumber += 1
			m = numpy.median(oneColumn, overwrite_input=True)
			medians[d] = m
		return medians

	def ComputeDistanceHistogram(self, fp = sys.stdout):
		'''Calculate the nearest-neighbor and any-neighbor distance
		histograms needed for the LSH Parameter Optimization.  For
		a number of random query points, print the distance to the
		nearest neighbor, and to any random neighbor.  This becomes
		the input for the parameter optimization routine.  Enhanced
		to also print the NN binary projections.'''
		numPoints = self.NumPoints()
		# medians = self.FindMedian()		# Not used now, but useful for binary quantization
		print("Pulling %d items from the NearestNeighbors list for ComputeDistanceHistogram" % \
			len(self.nearestNeighbors.items()))
		for (queryKey,(nnKey,nnDist)) in self.nearestNeighbors.items():
			randKey = self.GetRandomQuery()

			queryData = self.RetrieveData(queryKey)
			nnData = self.RetrieveData(nnKey)
			randData = self.RetrieveData(randKey)
			if len(queryData) == 0 or len(nnData) == 0:			# Missing, probably because of subsampling
				print("Skipping %s/%s because data is missing." % (queryKey, nnKey))
				continue
			anyD2 = ((randData-queryData)**2).sum()

			projection = numpy.random.randn(1, queryData.shape[0])
			# print "projection:", projection.shape
			# print "queryData:", queryData.shape
			# print "nnData:", nnData.shape
			# print "randData:", randData.shape
			queryProj = numpy.sign(numpy.dot(projection, queryData))
			nnProj = numpy.sign(numpy.dot(projection, nnData))
			randProj = numpy.sign(numpy.dot(projection, randData))

			# print 'CDH:', queryProj, nnProj, randProj
			fp.write('%g %g %d %d\n' % \
				(nnDist, math.sqrt(anyD2), \
				 queryProj==nnProj, queryProj==randProj))
			fp.flush()

	def ComputePnnPany(self, w, k, l, multiprobe=0):
		'''Compute the probability of Pnn and Pany for a given index size.
		Create the desired index, populate it with the data, and then measure
		the NN and ANY neighbor retrieval rates.
		Return
			the pnn rate for one 1-dimensional index (l=1),
			the pnn rate for an l-dimensional index,
			the pany rate for one 1-dimensional index (l=1),
			and the pany rate for an l-dimensional index
			the CPU time per query (seconds)'''
		numPoints = self.NumPoints()
		numDims = self.NumDimensions()
		self.CreateIndex(w, k, l)			# Put data into new index
		cnn  = 0; cnnFull  = 0
		cany = 0; canyFull = 0
		queryCount = 0							# Probe the index
		totalQueryTime = 0
		startRecallTestTime = time.clock()
		# print "ComputePnnPany: Testing %d nearest neighbors." % len(self.nearestNeighbors.items())
		for (queryKey,(nnKey,dist)) in self.nearestNeighbors.items():
			queryData = self.RetrieveData(queryKey)
			if queryData == None or len(queryData) == 0:
				print("Can't find data for key %s" % str(queryKey))
				sys.stdout.flush()
				continue
			startQueryTime = time.clock()	# Measure CPU time
			matches = self.myIndex.Find(queryData, multiprobe)
			totalQueryTime += time.clock() - startQueryTime
			for (m,c) in matches:
				if nnKey == m:				# See if NN was found!!!
					cnn += c
					cnnFull += 1
				if m != queryKey:			# Don't count the query
					cany += c
			canyFull += len(matches)-1		# Total candidates minus 1 for query
			queryCount += 1
			# Some debugging for k curve.. print individual results
			# print "ComputePnnPany Debug:", w, k, l, len(matches), numPoints, cnn, cnnFull, cany, canyFull
		recallTestTime = time.clock() - startRecallTestTime
		print("Tested %d NN queries in %g seconds." % (queryCount, recallTestTime))
		sys.stdout.flush()
		if queryCount == 0:
			queryCount = 1					# To prevent divide by zero
		perQueryTime = totalQueryTime/queryCount
		print("CPP:", cnn, cnnFull, cany, canyFull)
		print("CPP:",  cnn/float(queryCount*l), cnnFull/float(queryCount), \
			cany/float(queryCount*l*numPoints), canyFull/float(queryCount*numPoints), \
			perQueryTime, numDims)
		return cnn/float(queryCount*l), cnnFull/float(queryCount), \
			cany/float(queryCount*l*numPoints), canyFull/float(queryCount*numPoints), \
			perQueryTime, numDims

	def ComputePnnPanyCurve(self, wList = .291032, multiprobe=0):
			if type(wList) == float or type(wList) == int:
				wList = [wList*10**((i-10)/10.0) for i in range(0,21)]
			for w in wList:
				(pnn, pnnFull, pany, panyFull, queryTime, numDims) = self.ComputePnnPany(w, 1, 10, multiprobe)
				if w == wList[0]:
					print("# w pnn pany queryTime")
				print("PnnPany:", w, multiprobe, pnn, pany, queryTime)
				sys.stdout.flush()

	def ComputeKCurve(self, kList, w = .291032, r=0):
		'''Compute the number of ANY neighbors as a function of
		k.  Should go down exponentially.'''
		numPoints = self.NumPoints()
		l = 10
		for k in sorted(list(kList)):
			(pnn, pnnFull, pany, panyFull, queryTime, numDims) = self.ComputePnnPany(w, k, l, r)
			print(w, k, l, r, pnn, pany, pany*numPoints, queryTime)
			sys.stdout.flush()

	def ComputeLCurve(self, lList, w = 2.91032, k=10, r=0):
		'''Compute the probability of nearest neighbors as a function
		of l.'''
		numPoints = self.NumPoints()
		firstTime = True
		for l in sorted(list(lList)):
			(pnn, pnnFull, pany, panyFull, queryTime, numDims) = self.ComputePnnPany(w, k, l, r)
			if firstTime:
				print("# w k l r pnnFull, panyFull panyFull*N queryTime")
				firstTime = False
			print(w, k, l, r, pnnFull, panyFull, panyFull*numPoints, queryTime)
			sys.stdout.flush()



class RandomTestData(TestDataClass):
	'''Generate uniform random data points between -1 and 1.'''
	def CreateData(self, numPoints, dim):
		self.myData = (numpy.random.rand(dim, numPoints)-.5)*2.0

class HyperCubeTestData(TestDataClass):
	'''Create a hypercube of data.  All points are in the corners'''
	def CreateData(self, numDim, noise = None):
		numPoints = 2**numDim
		self.myData = numpy.zeros((numPoints, numDim))
		for i in range(0,numPoints):
			for b in range(0,numDim):
				if (2**b) & i:
					self.myData[b, i] = 1.0
		if noise != None:
			self.myData += (numpy.random.rand(numDim, numPoints)-.5)*noise

class RegularTestData(TestDataClass):
	'''Fill the 2-D test array with a regular grid of points between -1 and 1'''
	def CreateData(self, numDivs):
		self.myData = numpy.zeros(((2*numDivs+1)**2,2))
		i = 0
		for x in range(-numDivs, numDivs+1):
			for y in range(-numDivs, numDivs+1):
				self.myData[0, i] = x/float(divs)
				self.myData[1, i] = y/float(divs)
				i += 1



# Use Dimension Doubling to measure the dimensionality of a random
# set of data.  Generate some data (either random Gaussian or a grid)
# Then count the number of points that fall within the given radius of this
# query.
def XXXTestDimensionality2():
	binWidth = .5
	if True:
		numPoints = 100000
		myTestData = TestDataClass(numPoints, 3)
	else:
		myTestData = RegularTestData(100)
		numPoints = myTestData.NumPoints
	k = 4; l = 2; N = 1000
	myTestIndex = index(binWidth, k, l, N)
	for i in range(0,numPoints):
		myTestIndex.InsertIntoTable(i, myTestData.RetrieveData(i))
	rBig = binWidth/8.0
	rSmall = rBig/2.0
	cBig = 0.0; cSmall = 0.0
	for id in random.sample(ind.GetAllIndices(), 2):
		qp = FindLSHTestData(id)
		cBig += myTestIndex.CountInsideRadius(qp, myTestData.FindData, rBig)
		cSmall += myTestIndex.CountInsideRadius(qp, myTestData.FindData, rSmall)
	if cBig > cSmall and cSmall > 0:
		dim = math.log(cBig/cSmall)/math.log(rBig/rSmall)
	else:
		dim = 0
	print(cBig, cSmall, dim)
	return ind


# Generate some 2-dimensional data, put it into an index and then
# show the points retrieved.  This is all done as a function of number
# of projections per bucket, number of buckets to use for each index, and
# the number of LSH bucket (the T1 size).  Write out the data so we can
# plot it (in Matlab)
def GraphicalTest(k, l, N):
	numPoints = 1000
	myTestData = TestDataClass(numPoints, 3)
	ind = index(.1, k, l, N)
	for i in range(0,numPoints):
		ind.InsertIntoTable(i, myTestData.RetrieveData(i))
	i = 42
	r = ind.Find(data[i,:])
	fp = open('lshtestpoints.txt','w')
	for i in range(0,numPoints):
		if i in r:
			c = r[i]
		else:
			c = 0
		fp.write("%g %g %d\n" % (data[i,0], data[i,1], c))
	fp.close()
	return r



def SimpleTest():
	import time
	dim = 250
	numPoints = 10000
	myTestData = RandomTestData()
	myTestData.CreateData(numPoints,dim)
	myTestIndex = index(w=.4, k=10, l=10, N=numPoints)
	startLoad = time.clock()
	for id in myTestData.IterateKeys():
		data = myTestData.RetrieveData(id)
		myTestIndex.InsertIntoTable(id, data)
	endLoad = time.clock()
	print("Time to load %d points is %gs (%gms per point)" % \
		(numPoints, endLoad-startLoad, (endLoad-startLoad)/numPoints*1000.0))

	startRecall = time.clock()
	resCount = 0
	resFound = 0
	for id in myTestData.IterateKeys():
		query = myTestData.RetrieveData(id)
		res = myTestIndex.Find(query)
		if not res == None and len(res) > 0:
			resFound += 1
		if not res == None:
			resCount += len(res)
	endRecall = time.clock()
	print("Time to recall %d points is %gs (%gms per point" % \
		(numPoints, endRecall-startRecall, (endRecall-startRecall)/numPoints*1000.0))
	print("Found a recall hit all but %d times, average results per query is %g" % \
		(numPoints-resFound, resCount/float(numPoints)))




def OutputAllProjections(myTestData, myTestIndex, filename):
	'''Calculate and output all the projected data for an index.'''
	lshProjector = myTestIndex.projections[0]
	fp = open(filename, 'w')
	for id in myTestData.IterateKeys():
		d = myTestData.RetrieveData(id)
		(t1, t2, bins, parray) = lshProjector.CalculateHashes2(d)
		fp.write('%d %d %g %g\n' % (t1, t2, bins[0][0], parray[0][0]))
	fp.close()

# 	Exact Optimization:
#		For 100000 5-d data use: w=2.91032 and get 0.55401 hits per bin and 0.958216 nn.
#			K=23.3372 L=2.70766 cost is 2.98756
#	Expected statistics for optimal solution:
#		Assuming K=23, L=3
#		p_nn(w) is 0.958216
#		p_any(w) is 0.55401
#		Probability of finding NN for L=1: 0.374677
#		Probability of finding ANY for L=1: 1.26154e-06
#		Probability of finding NN for L=3: 0.75548
#		Probability of finding ANY for L=3: 3.78462e-06
#		Expected number of hits per query: 0.378462

'''
10-D data:
Mean of Python NN data is 0.601529 and std is 0.0840658.
Scaling all distances by 0.788576 for easier probability calcs.
Simple Approximation:
	For 100000 5-d data use: w=4.17052 and get 0.548534 hits per bin and 0.885004 nn.
		K=19.172 L=10.4033 cost is 20.8065
Expected statistics: for simple approximation
	Assuming K=19, L=10
	Probability of finding NN for L=1: 0.0981652
	Probability of finding ANY for L=1: 1.10883e-05
	Probability of finding NN for L=10: 0.644148
	Probability of finding ANY for L=10: 0.000110878
	Expected number of hits per query: 11.0878
Exact Optimization:
	For 100000 5-d data use: w=4.26786 and get 0.556604 hits per bin and 0.887627 nn.
		K=21.4938 L=12.9637 cost is 17.3645
Expected statistics for optimal solution:
	Assuming K=21, L=13
	p_nn(w) is 0.887627
	p_any(w) is 0.556604
	Probability of finding NN for L=1: 0.0818157
	Probability of finding ANY for L=1: 4.53384e-06
	Probability of finding NN for L=13: 0.670323
	Probability of finding ANY for L=13: 5.89383e-05
	Expected number of hits per query: 5.89383
'''

if __name__ == '__main__':
	defaultDims = 10
	defaultW = 2.91032
	defaultK = 10
	defaultL = 1
	defaultClosest = 1000
	defaultMultiprobeRadius = 0
	defaultFileName = 'testData'
	cmdName = sys.argv.pop(0)
	while len(sys.argv) > 0:
		arg = sys.argv.pop(0).lower()
		if arg == '-d':
			arg = sys.argv.pop(0)
			try:
				defaultDims = int(arg)
				defaultFileName = 'testData%03d' % defaultDims
			except:
				print("Couldn't parse new value for defaultDims: %s" % arg)
			print('New default dimensions for test is', defaultDims)
		elif arg == '-f':
			defaultFileName = sys.argv.pop(0)
			print('New file name is', defaultFileName)
		elif arg == '-k':
			arg = sys.argv.pop(0)
			try:
				defaultK = int(arg)
			except:
				print("Couldn't parse new value for defaultK: %s" % arg)
			print('New default k for test is', defaultK)
		elif arg == '-l':
			arg = sys.argv.pop(0)
			try:
				defaultL = int(arg)
			except:
				print("Couldn't parse new value for defaultL: %s" % arg)
			print('New default l for test is', defaultL)
		elif arg == '-c':
			arg = sys.argv.pop(0)
			try:
				defaultClosest = int(arg)
			except:
				print("Couldn't parse new value for defaultClosest: %s" % arg)
			print('New default number closest for test is', defaultClosest)
		elif arg == '-w':
			arg = sys.argv.pop(0)
			try:
				defaultW = float(arg)
			except:
				print("Couldn't parse new value for w: %s" % arg)
			print('New default W for test is', defaultW)
		elif arg == '-r':
			arg = sys.argv.pop(0)
			try:
				defaultMultiprobeRadius = int(arg)
			except:
				print("Couldn't parse new value for multiprobeRadius: %s" % arg)
			print('New default multiprobeRadius for test is', defaultMultiprobeRadius)
		elif arg == '-create':			# Create some uniform random data and find NN
			myTestData = RandomTestData()
			myTestData.CreateData(100000, defaultDims)
			myTestData.SaveData(defaultFileName + '.dat')
			print("Finished creating random data.  Now computing nearest neighbors...")
			myTestData.FindNearestNeighbors(defaultClosest)
			myTestData.SaveNearestNeighbors(defaultFileName + '.nn')
		elif arg == '-histogram':		# Calculate distance histograms
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			fp = open(defaultFileName + '.distances', 'w')
			if fp:
				myTestData.ComputeDistanceHistogram(fp)
				fp.close()
			else:
				print("Can't open %s.distances to store NN data" % defaultFileName)
		elif arg == '-sanity':
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			print(myTestData.RetrieveData(myTestData.GetRandomQuery()))
			print(myTestData.RetrieveData(myTestData.GetRandomQuery()))
		elif arg == '-b':		# Calculate bucket probabilities
			random.seed(0)
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			# ComputePnnPanyCurve(myData, [.291032])
			myTestData.ComputePnnPanyCurve(defaultW)
		elif arg == '-wtest':		# Calculate bucket probabilities as a function of w
			random.seed(0)
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			wList = [defaultW*.5**-i for i in range(-10,10)]
			# wList = [defaultW*.5**-i for i in range(-3,3)]
			myTestData.ComputePnnPanyCurve(wList, defaultMultiprobeRadius)
		elif arg == '-ktest':		# Calculate bucket probabilities as a function of k
			random.seed(0)
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			# ComputePnnPanyCurve(myData, [.291032])
			kList = [math.floor(math.sqrt(2)**k) for k in range(0,10)]
			kList = [1,2,3,4,5,6,8,10,12,14,16,18,20,22,25,30,35,40]
			myTestData.ComputeKCurve(kList, defaultW, defaultMultiprobeRadius)
		elif arg == '-ltest':		# Calculate bucket probabilities as a function of l
			random.seed(0)
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			# ComputePnnPanyCurve(myData, [.291032])
			lList = [math.floor(math.sqrt(2)**k) for k in range(0,10)]
			lList = [1,2,3,4,5,6,8,10,12,14,16,18,20,22,25,30]
			myTestData.ComputeLCurve(lList, w=defaultW,
				k=defaultK, r=defaultMultiprobeRadius)
		elif arg == '-timing':
			# sys.argv.pop(0)
			timingModels = []
			while len(sys.argv) > 0:
				print("Parsing timing argument", sys.argv[0], len(sys.argv))
				if sys.argv[0].startswith('-'):
					break
				try:
					(w,k,l,r,rest) = sys.argv[0].strip().split(',', 5)
					timingModels.append([float(w), int(k), int(l), int(r)])
				except:
					print("Couldn't parse %s.  Need w,k,l,r" % sys.argv[0])
				sys.argv.pop(0)
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			for (w, k, l, r) in timingModels:
				sys.stdout.flush()
				(pnnL1, pnn, panyL1, pany, perQueryTime, numDims) = myTestData.ComputePnnPany(w, k, l, r)
				print("Timing:", w, k, l, r, myTestData.NumPoints(), pnn, pany, perQueryTime*1000.0, numDims)

		elif arg == '-test':		# Calculate bucket probabilities as a function of l
			random.seed(0)
			myTestData = TestDataClass()
			myTestData.LoadData(defaultFileName + '.dat')
			myTestData.LoadNearestNeighbors(defaultFileName + '.nn')
			# ComputePnnPanyCurve(myData, [.291032])
			myTestData.ComputeLCurve([defaultL], w=defaultW, k=defaultK)
		else:
			print('%s: Unknown test argument %s' % (cmdName, arg))

New file name is /root/.local/share/jupyter/runtime/kernel-1c195346-6f0b-4a4f-9b23-1ad30712fbab.json


### LSH Blocking

In [0]:
#GiG

import csv
import torchfile
import numpy
import random
import time
import sys


#To ensure reproducibility - feel free to change them
# Use 20 random seeds as LSH is very probabilistic
# the values are got by sorted([random.randint(0, 100000) for gig in range(20)])
#random_seeds = [778, 3527, 3647, 6917, 12539, 12684, 16778, 19300, 27564, 42453, 47877, 50564, 55767, 60546, 60885, 66032, 71986, 90605, 95046, 97852]
random_seeds = [778, 3527, 3647, 6917, 12539, 12684, 16778, 19300, 27564, 42453]



def get_csv_reader(csv_file_name):
    f = open(csv_file_name)
    reader = csv.reader(f)
    next(reader)
    return reader

torch_id_to_vec_hash = {}

def id_to_vec(id_val):
    return torch_id_to_vec_hash[id_val]

def get_torch_data_as_dicts(torch_file_name):
    global torch_id_to_vec_hash

    data = torchfile.load(torch_file_name)
    dict_id_to_index = {}
    dict_vec_to_id = {}
    for i in range(len(data)):
        id_val = data[i]['id']
        dict_id_to_index[id_val] = i
        torch_id_to_vec_hash[id_val] = data[i]['vec']
    return data, dict_id_to_index

#No need to index both datasets
#Index the smaller one and query the bigger one
def index_data(lsh_engine, data, dataset1_size, dimension=300):
    for i in range(dataset1_size):
        #Sara: the following doesnt seem to work after some numpy update
        #vector = data[i]['vec'].reshape((dimension, 1))
        vector = data[i]['vec']
        vector.shape = (dimension, 1)
        id_val = data[i]['id']
        lsh_engine.InsertIntoTable(id_val, vector)

def create_lsh(K, L):
    #create L projects of size K each
    lsh_engine = lsh.index(float('inf'), K, L)
    return lsh_engine

def query_data_binary(lsh_engine, query_vec, match_id):
    matches = lsh_engine.Find(query_vec)
    tuple_ids = [elem[0] for elem in matches]
    return match_id in tuple_ids

def query_data_non_binary(lsh_engine, query_vec, match_id, topK=None, multi_probe=False):
    #old code that gives a distance proxy which is the number of buckets in which the item fell into same bucket as query vector
    #if multi_probe:
    #    matches = lsh_engine.FindMP(query_vec, 2)
    #else:
    #    matches = lsh_engine.Find(query_vec)
    multi_probe_radius = 0
    if multi_probe:
        multi_probe_radius = 2
    matches = lsh_engine.FindExact(query_vec, id_to_vec, multi_probe_radius)
    if topK is None:
        tuple_ids = [elem[0] for elem in matches]
    else:
        #Old code
        #sorted_matches = sorted(matches, key=lambda x: x[1], reverse=True)
        sorted_matches = sorted(matches, key=lambda x: x[1])
        tuple_ids = [elem[0] for elem in matches[:topK]]
    return match_id in tuple_ids, len(set(tuple_ids))

def get_vector(data, dict_id_to_index, index, dimension=300):
    vector_index = dict_id_to_index[index]
    vector = data[vector_index]['vec']
    vector = vector.reshape((dimension, 1))
    return vector

def compute_recall(lsh_engine, data, ground_truth_mapping, dict_id_to_index, topK=None, multi_probe=False):
    total_duplicates = 0
    found_duplicates = 0
    total_comparisons = 0
    for tuple_id1, tuple_id2 in ground_truth_mapping:
        vector2 = get_vector(data, dict_id_to_index, tuple_id2)
        result2, block_size = query_data_non_binary(lsh_engine, vector2, tuple_id1, topK, multi_probe)
        total_duplicates = total_duplicates + 1
        total_comparisons = total_comparisons + block_size
        if result2 is True:
            found_duplicates = found_duplicates + 1
    return float(found_duplicates) / float(total_duplicates), total_comparisons


def print_stats(torch_file_name, dataset1_size, dataset2_size):
    data = torchfile.load(torch_file_name)
    print(len(data), dataset1_size + dataset2_size)
    print(data[0]['id'], data[dataset1_size-1]['id'])
    print(data[dataset1_size]['id'], data[dataset1_size+dataset2_size-1]['id'], data[-1]['id'])


def test_abt_buy(K, L):
    lsh_engine = create_lsh(K, L)
    ground_truth_mapping = get_csv_reader ("abt_buy_perfectMapping.csv")
    data, dict_id_to_index  = get_torch_data_as_dicts("Abt-Buy.t7")
    dataset1_size = 1081
    dataset2_size = 1092
    index_data(lsh_engine, data, dataset1_size)
    recall, block_size  = compute_recall(lsh_engine, data, ground_truth_mapping, dict_id_to_index)
    return recall, block_size

def test_amzn_google(K,L, topK=None, multi_probe=False):
    lsh_engine = create_lsh(K, L)
    ground_truth_mapping = get_csv_reader ("Amzon_GoogleProducts_perfectMapping.csv")
    data, dict_id_to_index  = get_torch_data_as_dicts("Amazon-GoogleProducts.t7")
    dataset1_size = 1363
    dataset2_size = 3226
    index_data(lsh_engine, data, dataset1_size)
    recall, block_size  = compute_recall(lsh_engine, data, ground_truth_mapping, dict_id_to_index, topK, multi_probe)
    return recall, block_size

def test_dblp_acm(K,L):
    lsh_engine = create_lsh(K, L)
    ground_truth_mapping = get_csv_reader ("DBLP-ACM_perfectMapping.csv")
    data, dict_id_to_index  = get_torch_data_as_dicts("DBLP-ACM.t7")
    dataset1_size = 2616
    dataset2_size = 2294
    index_data(lsh_engine, data, dataset1_size)
    recall, block_size  = compute_recall(lsh_engine, data, ground_truth_mapping, dict_id_to_index, multi_probe)
    return recall, block_size

def test_dblp_scholar(K,L, topK=None, multi_probe=False):
    lsh_engine = create_lsh(K, L)
    ground_truth_mapping = get_csv_reader ("DBLP-Scholar_perfectMapping.csv")
    data, dict_id_to_index  = get_torch_data_as_dicts("DBLP-Scholar.t7")
    dataset1_size = 2616
    dataset2_size = 64263
    index_data(lsh_engine, data, dataset1_size)
    recall, block_size  = compute_recall(lsh_engine, data, ground_truth_mapping, dict_id_to_index, topK)
    return recall, block_size

def test_dataset(test_fn_name, K, L, topK=None, multi_probe=False):
    total_recall = 0
    total_comparisons = 0
    for random_seed in random_seeds:
        #Lsh module uses numpy for random numbers
        numpy.random.seed(random_seed)
        recall, block_size = test_fn_name(K, L, topK, multi_probe)
        #print K, L, random_seed, "Recall: ", recall
        total_recall = total_recall + recall
        total_comparisons = total_comparisons + block_size
    avg_recall = float(total_recall) / len(random_seeds)
    avg_comparisons = float(total_comparisons) / len(random_seeds)
    #print K, L, avg_recall
    return avg_recall, avg_comparisons

def test_dataset_diff_K_L(test_fn_name, op_file_name, dataset_name, range_K=10, range_L=10):
    with open(op_file_name, 'a') as csv_file:
        csv_writer = csv.writer(csv_file)
        for K in range(1, range_K+1):
            for L in range(1, range_L+1):
                avg_recall, avg_comparisons = test_dataset(test_fn_name, K, L)
                csv_writer.writerow([dataset_name, K, L, avg_recall, avg_comparisons])
                csv_file.flush()

def compute_K_L(n, P_1, P_2):
    inv_P1 = 1.0 / P_1
    inv_P2 = 1.0 / P_2

    K = int( numpy.ceil( numpy.log(n) /  numpy.log(inv_P2)))

    rho = numpy.log(inv_P1) / numpy.log(inv_P2)
    L = int( numpy.ceil( numpy.power(n, rho) ))
    return K, L


def test_dataset_vary_P2(dataset_name, test_fn_name, n, P_1 = 0.95):
    with open("vary_P2_fixed_P1.csv", "a") as csv_file:
        csv_writer = csv.writer(csv_file)

        for P_2 in [0.5, 0.6, 0.7, 0.8, 0.9]:
            K, L = compute_K_L(n, P_1, P_2)

            avg_recall, avg_comparisons = test_dataset(test_fn_name, K, L) #run LSH for 10 runs
            csv_writer.writerow([dataset_name, K, L, P_1, P_2, avg_recall, avg_comparisons])
            csv_file.flush()


def test_dataset_vary_P2_wrapper():
    amazn_google_size = 1363
    test_dataset_vary_P2("Amazon-Google", test_amzn_google, amazn_google_size)
    dblp_scholar_size = 2616
    test_dataset_vary_P2("DBLP-Scholar", test_dblp_scholar, dblp_scholar_size)


def test_topK_vs_recall(test_fn_name, dataset_name, op_file_name, K, L, multi_probe=False):
    with open(op_file_name, "a") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["Dataset name", "K", "L", "TopK", "Avg Recall", "Avg Comparisons", "MultiProbe"])
        for topK in [10, 20, 30, 40, 50, 100]:
            avg_recall, avg_comparisons = test_dataset(test_fn_name, K, L, topK, multi_probe)
            csv_writer.writerow([dataset_name, K, L, topK, avg_recall, avg_comparisons, multi_probe])
            csv_file.flush()

def test_topK_vs_recall_wrapper():
    K = 10
    L = 1
    #test_topK_vs_recall(test_amzn_google, "Amzn-Google", "op_topK_vs_Recall.csv", K, L, multi_probe=False)
    #test_topK_vs_recall(test_dblp_scholar, "DBLP-Scholar", "op_topK_vs_Recall.csv", K, L, multi_probe=False)
    test_topK_vs_recall(test_amzn_google, "Amzn-Google", "op_topK_vs_Recall.csv", K, L, multi_probe=True)
    #test_topK_vs_recall(test_dblp_scholar, "DBLP-Scholar", "op_topK_vs_Recall.csv", K, L, multi_probe=True)

def test_multi_probe_vs_recall_wrapper():
    test_multi_probe_vs_recall(test_amzn_google, "Amzn-Google", "op_multiprobe_vs_recall.csv")
    test_multi_probe_vs_recall(test_dblp_scholar, "DBLP-Scholar", "op_multiprobe_vs_recall.csv")


#print time.ctime()
#test_dataset_diff_K_L(test_abt_buy, "op_all_K_vs_L.csv", "Abt_Buy", 10, 10)
#print time.ctime()
#test_dataset_diff_K_L(test_amzn_google, "op_all_K_vs_L.csv", "Amzn_Google", 10, 10)
#print time.ctime()
#test_dataset_diff_K_L(test_dblp_acm, "op_all_K_vs_L.csv", "DBLP_ACM", 10, 10)
#print time.ctime()
#test_dataset_diff_K_L(test_dblp_scholar, "op_all_K_vs_L.csv", "DBLP_Scholar", 10, 10)
#print time.ctime()

#test_dataset_vary_P2_wrapper()

#test_topK_vs_recall_wrapper()

#test_multi_probe_vs_recall_wrapper()

### Process dataset using absolute vector difference

In [0]:
#GiG.

#This file mostly utility functions to process dataset files to the expected data format

import csv
import os
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
import fasttext
from scipy.spatial.distance import cosine
import torch

#This function splits an input file from a given path into three : train, validation and test
# then output files train.csv, validation.csv, test.csv in the folder folder_path
# conservatively, the split is done in a stratified manner by manually splitting data into duplicates and non duplicates
# this is relevant when we test very small number of training data

def split_dataset_by_ratio(folder_path, candset_ids_file_name, split_ratio=[0.3, 0.2, 0.5], label_field_name= 'gold', random_state=12345):
    df = pd.read_csv(os.path.join(folder_path, candset_ids_file_name), encoding="utf-8")
    duplicates_df = df[df[label_field_name] == 1]
    non_duplicates_df = df[df[label_field_name] == 0]

    train_duplicates, validation_duplicates, test_duplicates = local_train_validate_test_split(duplicates_df, split_ratio, random_state)
    train_non_duplicates, validation_non_duplicates, test_non_duplicates = local_train_validate_test_split(non_duplicates_df, split_ratio, random_state)

    #The last sample is to shuffle the data so that duplicates and non_duplicates mix
    train_df = pd.concat([train_duplicates, train_non_duplicates]).sample(frac=1)
    validation_df = pd.concat([validation_duplicates, validation_non_duplicates]).sample(frac=1)
    test_df = pd.concat([test_duplicates, test_non_duplicates]).sample(frac=1)

    #verify_split(df, train_df, validation_df, test_df, split_ratio, label_field_name)
    for partition_df, file_name in [(train_df, "train.csv"), (validation_df, "validation.csv"), (test_df, "test.csv")]:
        partition_df.to_csv(os.path.join(folder_path, file_name), encoding="utf-8", index=False)


def local_train_validate_test_split(df, split_ratio=[0.3, 0.2, 0.5], random_state=12345):
    np.random.seed(random_state)
    random_shuffle = np.random.permutation(df.index)
    num_tuples = len(df)

    train_end = int(num_tuples * split_ratio[0])
    validation_end = train_end + int(num_tuples * split_ratio[1])

    train_df = df.ix[random_shuffle[:train_end]]
    validation_df = df.ix[random_shuffle[train_end:validation_end]]
    test_df = df.ix[random_shuffle[validation_end:]]

    return train_df, validation_df, test_df

#Trivial manual validation to check correctness of the split
def verify_split(df, train_df, validation_df, test_df, split_ratio, label_field_name):
    num_duplicates_df = len(df[df[label_field_name] == 1])
    num_non_duplicates_df = len(df[df[label_field_name] == 0])

    for index, partition_df in enumerate([train_df, validation_df, test_df]):
        num_partition_duplicates = len(partition_df[partition_df[label_field_name] == 1])
        num_partition_non_duplicates = len(partition_df[partition_df[label_field_name] == 0])
        expected_duplicates, expected_non_duplicates = int(num_duplicates_df * split_ratio[index]), int(num_non_duplicates_df * split_ratio[index])
        actual_duplicates, actual_non_duplicates = num_partition_duplicates, num_partition_non_duplicates
        if actual_duplicates != expected_duplicates or actual_non_duplicates != expected_non_duplicates:
            print("Mismatch :", expected_duplicates, actual_duplicates, expected_non_duplicates, actual_non_duplicates)

#Given a set of (ltable_id, rtable_id, gold) triples,
# construct the distributional similarity vector and store it
#First is the folder, second is the train|validation|test file, and the last two are the csv files of the left and right datasets
#Output file name is obtained from input_file_name and is put in the same folder as folder_path
# This is not very efficient - partially to avoid storing large intermediate matrices
def dataset_to_matrix(folder_path, input_file_name, ltable_file_name, rtable_file_name):
    ltable_df = pd.read_csv( os.path.join(folder_path, ltable_file_name), encoding="utf-8")
    rtable_df = pd.read_csv( os.path.join(folder_path, rtable_file_name), encoding="utf-8")

    candset_with_ids_df = pd.read_csv( os.path.join(folder_path, input_file_name), encoding="utf-8" )

    #Find common attributes of ltable and rtable
    common_attributes = ltable_df.columns.intersection(rtable_df.columns)
    #Only keep the common attributes
    ltable_df = ltable_df[common_attributes]
    rtable_df = rtable_df[common_attributes]

    #Assumption id column is titled as "id"
    #Remove this from nltk processing
    common_attributes = list(common_attributes.drop("id"))
    ltable_df = ltable_df.fillna(" ")
    rtable_df = rtable_df.fillna(" ")

    #This is a list of punctuations to remove with empty string
    replacement_list = get_replacement_list()

    for attribute in common_attributes:
        ltable_df[attribute] = ltable_df[attribute].replace(regex=replacement_list, value= " ")
        rtable_df[attribute] = rtable_df[attribute].replace(regex=replacement_list, value= " ")

    #Assumes key column name is id. Creates a dictionary where key is the id and the value is row index
    id_to_idx_dict_ltable = dict(zip(ltable_df.id, ltable_df.index))
    id_to_idx_dict_rtable = dict(zip(rtable_df.id, rtable_df.index))

    #First m attributes are for cosine diff and next m for norm of abs diff
    #The last one is for gold label
    num_attributes = len(common_attributes)
    dist_repr_similarity_matrix = np.zeros( (len(candset_with_ids_df), DR_DIMENSION * num_attributes + 1), dtype=np.float32)

    #The following is an inefficient way to construct distributional similarity vectors
    # but avoids the need to create huge distributional representations for input vectors
    fasttext_model = load_fasttext_model()
    for row_index, row in candset_with_ids_df.iterrows():
        ltable_id, rtable_id, gold = row

        #Get the index of the current (ltable_id, rtable_id, gold) being processed
        ltable_index = id_to_idx_dict_ltable[ row['ltable_id'] ]
        rtable_index = id_to_idx_dict_rtable[ row['rtable_id'] ]

        #Get the corresponding rows
        ltable_row = ltable_df.iloc[ltable_index]
        rtable_row = rtable_df.iloc[rtable_index]

        for col_index, attribute in enumerate(common_attributes):
            abs_diff = compute_distance_abs_diff(fasttext_model, ltable_row[attribute], rtable_row[attribute])
            start_pos = col_index * DR_DIMENSION
            end_pos = (col_index+1) * DR_DIMENSION
            dist_repr_similarity_matrix[row_index][start_pos:end_pos] = abs_diff
            dist_repr_similarity_matrix[row_index][-1] = gold

    np.save(os.path.join(folder_path, input_file_name), dist_repr_similarity_matrix)

#This function takes two strings, converts to utf-8, computes their cosine and absolute error distance
def compute_distance(fasttext_model, ltable_str, rtable_str):
    if isinstance(ltable_str, basestring) == False:
        ltable_str = unicode(ltable_str)
    if isinstance(rtable_str, basestring) == False:
        rtable_str = unicode(rtable_str)

    lcol_dr = fasttext_model.get_sentence_vector(ltable_str)
    rcol_dr = fasttext_model.get_sentence_vector(rtable_str)

    #See the fillna command before for handling nulls
    if ltable_str == rtable_str and ltable_str == " ":
        #If both empty return lowest distance
        return 0.0, 0.0
    if ltable_str == " " and rtable_str != " ":
        return 1.0, 1.0
    if ltable_str != " " and rtable_str == " ":
        return 1.0, 1.0


    cosine_dist = cosine(lcol_dr, rcol_dr)
    normed_abs_dist = np.linalg.norm(np.abs(lcol_dr - rcol_dr))
    return cosine_dist, normed_abs_dist

#This function takes two strings, converts to utf-8, computes their cosine and absolute error distance
def compute_distance_abs_diff(fasttext_model, ltable_str, rtable_str):
    if isinstance(ltable_str, basestring) == False:
        ltable_str = unicode(ltable_str)
    if isinstance(rtable_str, basestring) == False:
        rtable_str = unicode(rtable_str)

    lcol_dr = fasttext_model.get_sentence_vector(ltable_str)
    rcol_dr = fasttext_model.get_sentence_vector(rtable_str)

    return np.abs(lcol_dr - rcol_dr)

#This is a helper function to create distributional similarity matrix for all datasets
# and calls the dist similarity computation for each of train, validation and test files
def compute_dist_similarity_matrix_wrapper():
    all_datasets = er_dataset_details.keys()
    for dataset_name in all_datasets:

        dataset = er_dataset_details[dataset_name]
        folder = dataset["dataset_folder_path"]
        input_file = "candset_ids_only.csv"
        ltable_file_name = dataset["ltable_file_name"]
        rtable_file_name = dataset["rtable_file_name"]

        #dataset_to_matrix("/Users/neo/Desktop/QCRI/DataCleaning/datasets/BenchmarkDatasets/Fodors_Zagat/", "candset_ids_only.csv", "fodors.csv", "zagats.csv")
        #dataset_to_matrix(folder, "candset_ids_only.csv", ltable_file, rtable_file)

        print("Processing ", dataset_name, " train.csv")
        dataset_to_matrix(folder, "train.csv", ltable_file_name, rtable_file_name)
        print("Processing ", dataset_name, " validation.csv")
        dataset_to_matrix(folder, "validation.csv", ltable_file_name, rtable_file_name)
        print("Processing ", dataset_name, " test.csv")
        dataset_to_matrix(folder, "test.csv", ltable_file_name, rtable_file_name)

def convert_csv_to_features(dataset_name, input_file_name):
    dataset = er_dataset_details[dataset_name]
    folder_path = dataset["dataset_folder_path"]
    ltable_file_name = dataset["ltable_file_name"]
    rtable_file_name = dataset["rtable_file_name"]

    feature_file_name = input_file_name.replace(".csv", "_abs_diff.npy")

    #Check if the npy file already exists
    file_path = os.path.join(folder_path, feature_file_name)
    if os.path.exists(file_path):
        print("File {} already exists. Reusing it.".format(feature_file_name))
    else:
        print("File {} does not exist. Creating and persisting it.".format(feature_file_name))
        dataset_to_matrix(folder_path, input_file_name, ltable_file_name, rtable_file_name, feature_file_name)
    return np.load(file_path)

def get_features_and_labels(dataset_name, file_name):
    matrix = convert_csv_to_features(dataset_name, file_name)
    features, labels = matrix[:, :-1], matrix[:, -1]

    #Convert to torch format from numpy format
    features, labels = torch.from_numpy(features), torch.from_numpy(labels).type(torch.LongTensor)
    return features, labels


#Function to efficiently remove common punctuation and stopwords
def get_replacement_list():
    replacement_list = [r'\.', r"\'", r'\"', r'\(', '\)', r'\,', r'\&', r'\\', r'\/']
    #for stopword in stopwords.words('english'):
    #    replacement_list.append(r'\b%s\b' % stopword)
    return replacement_list

def load_fasttext_model():
    return fastText.load_model(FASTTEXT_MODEL_PATH)

def get_folder_to_persist_model(dataset_name):
    dataset = er_dataset_details[dataset_name]
    folder = dataset["dataset_folder_path"]
    return folder

#if __name__ == "__main__":
#    compute_dist_similarity_matrix_wrapper()

### Process dataset using similarity

In [0]:
#GiG.

#This file mostly utility functions to process dataset files to the expected data format

import csv
import os
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
import fasttext
from scipy.spatial.distance import cosine
import torch

#This function splits an input file from a given path into three : train, validation and test
# then output files train.csv, validation.csv, test.csv in the folder folder_path
# conservatively, the split is done in a stratified manner by manually splitting data into duplicates and non duplicates
# this is relevant when we test very small number of training data

def split_dataset_by_ratio(folder_path, candset_ids_file_name, split_ratio=[0.3, 0.2, 0.5], label_field_name= 'gold', random_state=12345, train_file_name="train.csv", validation_file_name="validation.csv", test_file_name="test.csv"):
    df = pd.read_csv(os.path.join(folder_path, candset_ids_file_name), encoding="utf-8")
    duplicates_df = df[df[label_field_name] == 1]
    non_duplicates_df = df[df[label_field_name] == 0]

    train_duplicates, validation_duplicates, test_duplicates = local_train_validate_test_split(duplicates_df, split_ratio, random_state)
    train_non_duplicates, validation_non_duplicates, test_non_duplicates = local_train_validate_test_split(non_duplicates_df, split_ratio, random_state)

    #The last sample is to shuffle the data so that duplicates and non_duplicates mix
    train_df = pd.concat([train_duplicates, train_non_duplicates]).sample(frac=1)
    validation_df = pd.concat([validation_duplicates, validation_non_duplicates]).sample(frac=1)
    test_df = pd.concat([test_duplicates, test_non_duplicates]).sample(frac=1)

    #verify_split(df, train_df, validation_df, test_df, split_ratio, label_field_name)
    for partition_df, file_name in [(train_df, train_file_name), (validation_df, validation_file_name), (test_df, test_file_name)]:
        partition_df.to_csv(os.path.join(folder_path, file_name), encoding="utf-8", index=False)


def local_train_validate_test_split(df, split_ratio=[0.3, 0.2, 0.5], random_state=12345):
    np.random.seed(random_state)
    random_shuffle = np.random.permutation(df.index)
    num_tuples = len(df)

    train_end = int(num_tuples * split_ratio[0])
    validation_end = train_end + int(num_tuples * split_ratio[1])

    train_df = df.ix[random_shuffle[:train_end]]
    validation_df = df.ix[random_shuffle[train_end:validation_end]]
    test_df = df.ix[random_shuffle[validation_end:]]

    return train_df, validation_df, test_df

#Trivial manual validation to check correctness of the split
def verify_split(df, train_df, validation_df, test_df, split_ratio, label_field_name):
    num_duplicates_df = len(df[df[label_field_name] == 1])
    num_non_duplicates_df = len(df[df[label_field_name] == 0])

    for index, partition_df in enumerate([train_df, validation_df, test_df]):
        num_partition_duplicates = len(partition_df[partition_df[label_field_name] == 1])
        num_partition_non_duplicates = len(partition_df[partition_df[label_field_name] == 0])
        expected_duplicates, expected_non_duplicates = int(num_duplicates_df * split_ratio[index]), int(num_non_duplicates_df * split_ratio[index])
        actual_duplicates, actual_non_duplicates = num_partition_duplicates, num_partition_non_duplicates
        if actual_duplicates != expected_duplicates or actual_non_duplicates != expected_non_duplicates:
            print("Mismatch :", expected_duplicates, actual_duplicates, expected_non_duplicates, actual_non_duplicates)

#Given a set of (ltable_id, rtable_id, gold) triples,
# construct the distributional similarity vector and store it
#First is the folder, second is the train|validation|test file, and the last two are the csv files of the left and right datasets
#Output file name is obtained from input_file_name and is put in the same folder as folder_path
# This is not very efficient - partially to avoid storing large intermediate matrices
def dataset_to_matrix(folder_path, input_file_name, ltable_file_name, rtable_file_name):
    ltable_df = pd.read_csv( os.path.join(folder_path, ltable_file_name), encoding="utf-8")
    rtable_df = pd.read_csv( os.path.join(folder_path, rtable_file_name), encoding="utf-8")

    candset_with_ids_df = pd.read_csv( os.path.join(folder_path, input_file_name), encoding="utf-8" )

    #Find common attributes of ltable and rtable
    common_attributes = ltable_df.columns.intersection(rtable_df.columns)
    #Only keep the common attributes
    ltable_df = ltable_df[common_attributes]
    rtable_df = rtable_df[common_attributes]

    #Assumption id column is titled as "id"
    #Remove this from nltk processing
    common_attributes = list(common_attributes.drop("id"))
    ltable_df = ltable_df.fillna(" ")
    rtable_df = rtable_df.fillna(" ")

    #This is a list of punctuations to remove with empty string
    replacement_list = get_replacement_list()

    for attribute in common_attributes:
        ltable_df[attribute] = ltable_df[attribute].replace(regex=replacement_list, value= " ")
        rtable_df[attribute] = rtable_df[attribute].replace(regex=replacement_list, value= " ")

    #Assumes key column name is id. Creates a dictionary where key is the id and the value is row index
    id_to_idx_dict_ltable = dict(zip(ltable_df.id, ltable_df.index))
    id_to_idx_dict_rtable = dict(zip(rtable_df.id, rtable_df.index))

    #First m attributes are for cosine diff and next m for norm of abs diff
    #The last one is for gold label
    num_attributes = len(common_attributes)
    dist_repr_similarity_matrix = np.zeros( (len(candset_with_ids_df), 2 * num_attributes + 1), dtype=np.float32)

    #The following is an inefficient way to construct distributional similarity vectors
    # but avoids the need to create huge distributional representations for input vectors
    fasttext_model = load_fasttext_model()
    for row_index, row in candset_with_ids_df.iterrows():
        ltable_id, rtable_id, gold = row

        #Get the index of the current (ltable_id, rtable_id, gold) being processed
        ltable_index = id_to_idx_dict_ltable[ row['ltable_id'] ]
        rtable_index = id_to_idx_dict_rtable[ row['rtable_id'] ]

        #Get the corresponding rows
        ltable_row = ltable_df.iloc[ltable_index]
        rtable_row = rtable_df.iloc[rtable_index]

        for col_index, attribute in enumerate(common_attributes):
            cosine_dist, normed_abs_dist = compute_distance(fasttext_model, ltable_row[attribute], rtable_row[attribute])
            dist_repr_similarity_matrix[row_index][col_index] = cosine_dist
            dist_repr_similarity_matrix[row_index][num_attributes+col_index] = normed_abs_dist
            dist_repr_similarity_matrix[row_index][-1] = gold

    output_file_name = input_file_name.replace(".csv", "")
    np.save(os.path.join(folder_path, output_file_name), dist_repr_similarity_matrix)

#This function takes two strings, converts to utf-8, computes their cosine and absolute error distance
def compute_distance(fasttext_model, ltable_str, rtable_str):
    if isinstance(ltable_str, basestring) == False:
        ltable_str = unicode(ltable_str)
    if isinstance(rtable_str, basestring) == False:
        rtable_str = unicode(rtable_str)

    lcol_dr = fasttext_model.get_sentence_vector(ltable_str)
    rcol_dr = fasttext_model.get_sentence_vector(rtable_str)

    #See the fillna command before for handling nulls
    if ltable_str == rtable_str and ltable_str == " ":
        #If both empty return lowest distance
        return 0.0, 0.0
    if ltable_str == " " and rtable_str != " ":
        return 1.0, 1.0
    if ltable_str != " " and rtable_str == " ":
        return 1.0, 1.0


    cosine_dist = cosine(lcol_dr, rcol_dr)
    normed_abs_dist = np.linalg.norm(np.abs(lcol_dr - rcol_dr))
    return cosine_dist, normed_abs_dist


#This is a helper function to create distributional similarity matrix for all datasets
# and calls the dist similarity computation for each of train, validation and test files
def compute_dist_similarity_matrix_wrapper(dataset_name, train_file_name="train.csv", validation_file_name="validation.csv", test_file_name="test.csv"):
    dataset = er_dataset_details[dataset_name]
    folder = dataset["dataset_folder_path"]
    input_file = "candset_ids_only.csv"
    ltable_file_name = dataset["ltable_file_name"]
    rtable_file_name = dataset["rtable_file_name"]

    #dataset_to_matrix("/Users/neo/Desktop/QCRI/DataCleaning/datasets/BenchmarkDatasets/Fodors_Zagat/", "candset_ids_only.csv", "fodors.csv", "zagats.csv")
    #dataset_to_matrix(folder, "candset_ids_only.csv", ltable_file, rtable_file)

    print("Processing ", dataset_name, train_file_name)
    dataset_to_matrix(folder, train_file_name, ltable_file_name, rtable_file_name)
    print("Processing ", dataset_name, validation_file_name)
    dataset_to_matrix(folder, validation_file_name, ltable_file_name, rtable_file_name)
    print("Processing ", dataset_name, test_file_name)
    dataset_to_matrix(folder, test_file_name, ltable_file_name, rtable_file_name)

def convert_csv_to_features(dataset_name, input_file_name):
    dataset = er_dataset_details[dataset_name]
    folder_path = dataset["dataset_folder_path"]
    ltable_file_name = dataset["ltable_file_name"]
    rtable_file_name = dataset["rtable_file_name"]

    feature_file_name = input_file_name.replace(".csv", ".npy")

    #Check if the npy file already exists
    file_path = os.path.join(folder_path, feature_file_name)
    if os.path.exists(file_path):
        print("File {} already exists. Reusing it.".format(feature_file_name))
    else:
        print("File {} does not exist. Creating and persisting it.".format(feature_file_name))
        dataset_to_matrix(folder_path, input_file_name, ltable_file_name, rtable_file_name)
    return np.load(file_path)

def get_features_and_labels(dataset_name, file_name):
    matrix = convert_csv_to_features(dataset_name, file_name)
    features, labels = matrix[:, :-1], matrix[:, -1]

    #Convert to torch format from numpy format
    features, labels = torch.from_numpy(features), torch.from_numpy(labels).type(torch.LongTensor)
    return features, labels


#Function to efficiently remove common punctuation and stopwords
def get_replacement_list():
    replacement_list = [r'\.', r"\'", r'\"', r'\(', '\)', r'\,', r'\&', r'\\', r'\/']
    #for stopword in stopwords.words('english'):
    #    replacement_list.append(r'\b%s\b' % stopword)
    return replacement_list

def load_fasttext_model():
    return fastText.load_model(FASTTEXT_MODEL_PATH)

def get_folder_to_persist_model(dataset_name):
    dataset = er_dataset_details[dataset_name]
    folder = dataset["dataset_folder_path"]
    return folder

#Sample code to create training, validation and testing files
#Change the parameters below as necessary
def temp_sample_split_code():
    dataset_name = "DBLP_Scholar"
    folder_path = er_dataset_details[dataset_name]["dataset_folder_path"]
    candset_ids_file_name = "candset_ids_only.csv"
    split_dataset_by_ratio(folder_path, candset_ids_file_name, split_ratio=[0.3, 0.2, 0.5], label_field_name= 'gold', random_state=12345, train_file_name="train.csv",           validation_file_name="validation.csv", test_file_name="test.csv")


### DeepER using similarity

In [29]:
#GiG.

import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim

from sklearn.metrics import f1_score
import os
import random
import numpy as np
import pandas as pd

#DL Specific configs
BATCH_SIZE = 16
MAX_EPOCHS = 32
LEARNING_RATE = 0.001
BETAS = (0.9, 0.99)
EPSILON = 1e-9
RANDOM_STATE = 12345
HIDDEN_X = 2
MODEL_FILE_NAME = "best_validation_model_params.torch"

def get_deeper_lite_model_sim(num_attributes):

    #If there are K input attributes, Deeper Lite  has 2K features : 1 each for cosine distance and normed abs distance
    #Hidden_X is a multiplicative factor controlling the size of hidden layer.
    deeper_lite_model = nn.Sequential(
        nn.Linear(int(2 * num_attributes), int(HIDDEN_X * num_attributes)),
        nn.ReLU(),
        nn.Linear(int(HIDDEN_X * num_attributes), int(HIDDEN_X * num_attributes)),
        nn.ReLU(),
        nn.Linear(int(HIDDEN_X * num_attributes), int(HIDDEN_X * num_attributes)),
        nn.ReLU(),
        nn.Linear(int(HIDDEN_X * num_attributes), 2),
    )

    return deeper_lite_model

#Assumes that the train and validation files are in the same folder as dataset_name
def train(dataset_name, train_file_name, validation_file_name, model_fn):
    train_features, train_labels = get_features_and_labels(dataset_name, train_file_name)
    validation_features, validation_labels = get_features_and_labels(dataset_name, validation_file_name)

    #Hack: Assumes that for deeper lite num_features = 2 * num_attributes
    num_attributes = int(train_features.shape[1] / 2)
    model = model_fn(num_attributes)

    train_dataset = Data.TensorDataset(train_features, train_labels)
    #Allows us to read the dataset in batches
    training_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=BETAS, eps=EPSILON)
    criterion = nn.CrossEntropyLoss()


    #For reproducibility
    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)
    torch.manual_seed(RANDOM_STATE)

    best_validation_f1_score = 0.0
    best_model_so_far = None
    model_file_name_path = os.path.join(get_folder_to_persist_model(dataset_name) , MODEL_FILE_NAME)

    for epoch in range(MAX_EPOCHS):
        for batch_idx, (train_features, train_labels) in enumerate(training_loader):
            optimizer.zero_grad()
            output = model(train_features)
            loss = criterion(output, train_labels)
            loss.backward()
            optimizer.step()

        training_f1_score = compute_scores(output, train_labels)
        with torch.no_grad():
            validation_output = model(validation_features)
            validation_f1_score = compute_scores(validation_output, validation_labels)
            if validation_f1_score > best_validation_f1_score:
                best_model_so_far = model.state_dict()
                best_validation_f1_score = validation_f1_score

    torch.save(best_model_so_far, model_file_name_path)
    print("Curr Val F1, Best Val F1 ", validation_f1_score, best_validation_f1_score)
    return best_model_so_far

def test(dataset_name, test_file_name, test_output_file_name, model_fn):
    test_features, test_labels = get_features_and_labels(dataset_name, test_file_name)
    #Hack: Assumes that for deeper lite num_features = 2 * num_attributes
    num_attributes = test_features.shape[1] / 2
    model = model_fn(num_attributes)

    folder_path = get_folder_to_persist_model(dataset_name)
    model_file_name_path = os.path.join( folder_path, MODEL_FILE_NAME)
    model.load_state_dict(torch.load(model_file_name_path))
    model.eval()

    predictions = model(test_features)
    #Uncomment the following lines to get the score
    #testing_f1_score = compute_scores(predictions, test_labels)
    #print "Testing F1 ", testing_f1_score

    prediction_as_numpy = torch.max(predictions, 1)[1].data.numpy()

    #Store output
    test_df = pd.read_csv( os.path.join(folder_path, test_file_name), encoding="utf-8" )
    test_df["gold"] = prediction_as_numpy
    test_df.to_csv(os.path.join(folder_path, test_output_file_name), encoding="utf8", index=False)

def compute_scores(predicted, actual):
    #Convert from cross entropy output to actual 0/1 predictions
    predicted = torch.max(predicted, 1)[1].data

    #Convert to numpy format
    predicted_numpy = predicted.numpy()
    actual_numpy = actual.numpy()

    #Print performance measures
    return f1_score(actual_numpy, predicted_numpy)

#if __name__ == "__main__":
train("Fodors_Zagat", "train.csv", "validation.csv", get_deeper_lite_model_sim)
test("Fodors_Zagat", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

    #train("Cora", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    #test("Cora", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

    #train("DBLP_ACM", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    #test("DBLP_ACM", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

    #train("DBLP_Scholar", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    #test("DBLP_Scholar", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

File train.npy already exists. Reusing it.
File validation.npy already exists. Reusing it.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Curr Val F1, Best Val F1  1.0 1.0
File test.npy already exists. Reusing it.


### DeepER using absolute vector difference

In [0]:
#GiG.

import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim

from sklearn.metrics import f1_score
import os
import random
import numpy as np
import pandas as pd

import process_dataset_sim as process_dataset

#DL Specific configs
BATCH_SIZE = 16
MAX_EPOCHS = 32
LEARNING_RATE = 0.001
BETAS = (0.9, 0.99)
EPSILON = 1e-9
RANDOM_STATE = 12345
HIDDEN_X = 2
MODEL_FILE_NAME = "best_validation_model_params.torch"

def get_deeper_lite_model_sim(num_attributes):

    #If there are K input attributes, Deeper Lite  has 2K features : 1 each for cosine distance and normed abs distance
    #Hidden_X is a multiplicative factor controlling the size of hidden layer.
    deeper_lite_model = nn.Sequential(
        nn.Linear(2 * num_attributes, HIDDEN_X * num_attributes),
        nn.ReLU(),
        nn.Linear(HIDDEN_X * num_attributes, HIDDEN_X * num_attributes),
        nn.ReLU(),
        nn.Linear(HIDDEN_X * num_attributes, HIDDEN_X * num_attributes),
        nn.ReLU(),
        nn.Linear(HIDDEN_X * num_attributes, 2),
    )

    return deeper_lite_model

#Assumes that the train and validation files are in the same folder as dataset_name
def train(dataset_name, train_file_name, validation_file_name, model_fn):
    train_features, train_labels = process_dataset.get_features_and_labels(dataset_name, train_file_name)
    validation_features, validation_labels = process_dataset.get_features_and_labels(dataset_name, validation_file_name)

    #Hack: Assumes that for deeper lite num_features = 2 * num_attributes
    num_attributes = train_features.shape[1] / 2
    model = model_fn(num_attributes)

    train_dataset = Data.TensorDataset(train_features, train_labels)
    #Allows us to read the dataset in batches
    training_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=BETAS, eps=EPSILON)
    criterion = nn.CrossEntropyLoss()


    #For reproducibility
    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)
    torch.manual_seed(RANDOM_STATE)

    best_validation_f1_score = 0.0
    best_model_so_far = None
    model_file_name_path = os.path.join( process_dataset.get_folder_to_persist_model(dataset_name) , MODEL_FILE_NAME)

    for epoch in range(MAX_EPOCHS):
        for batch_idx, (train_features, train_labels) in enumerate(training_loader):
            optimizer.zero_grad()
            output = model(train_features)
            loss = criterion(output, train_labels)
            loss.backward()
            optimizer.step()

        training_f1_score = compute_scores(output, train_labels)
        with torch.no_grad():
            validation_output = model(validation_features)
            validation_f1_score = compute_scores(validation_output, validation_labels)
            if validation_f1_score > best_validation_f1_score:
                best_model_so_far = model.state_dict()
                best_validation_f1_score = validation_f1_score

    torch.save(best_model_so_far, model_file_name_path)
    print "Curr Val F1, Best Val F1 ", validation_f1_score, best_validation_f1_score
    return best_model_so_far

def test(dataset_name, test_file_name, test_output_file_name, model_fn):
    test_features, test_labels = process_dataset.get_features_and_labels(dataset_name, test_file_name)
    #Hack: Assumes that for deeper lite num_features = 2 * num_attributes
    num_attributes = test_features.shape[1] / 2
    model = model_fn(num_attributes)

    folder_path = process_dataset.get_folder_to_persist_model(dataset_name)
    model_file_name_path = os.path.join( folder_path, MODEL_FILE_NAME)
    model.load_state_dict(torch.load(model_file_name_path))
    model.eval()

    predictions = model(test_features)
    #Uncomment the following lines to get the score
    #testing_f1_score = compute_scores(predictions, test_labels)
    #print "Testing F1 ", testing_f1_score

    prediction_as_numpy = torch.max(predictions, 1)[1].data.numpy()

    #Store output
    test_df = pd.read_csv( os.path.join(folder_path, test_file_name), encoding="utf-8" )
    test_df["gold"] = prediction_as_numpy
    test_df.to_csv(os.path.join(folder_path, test_output_file_name), encoding="utf8", index=False)

def compute_scores(predicted, actual):
    #Convert from cross entropy output to actual 0/1 predictions
    predicted = torch.max(predicted, 1)[1].data

    #Convert to numpy format
    predicted_numpy = predicted.numpy()
    actual_numpy = actual.numpy()

    #Print performance measures
    return f1_score(actual_numpy, predicted_numpy)

if __name__ == "__main__":
    train("Fodors_Zagat", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    test("Fodors_Zagat", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

    train("Cora", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    test("Cora", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

    train("DBLP_ACM", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    test("DBLP_ACM", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)

    train("DBLP_Scholar", "train.csv", "validation.csv", get_deeper_lite_model_sim)
    test("DBLP_Scholar", "test.csv", "test_predictions.csv", get_deeper_lite_model_sim)